In [1]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q3l/sgd_10q_3l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)

losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")


# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=config["nqubits"], delta=0.5)
circuit = build_circuit(config["nqubits"], config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(config["nqubits"])
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)


[Qibo 0.2.9|INFO|2024-06-06 13:44:32]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0
[Qibo 0.2.9|INFO|2024-06-06 13:44:33]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy


In [3]:
v_vqe = vqe.circuit.unitary()

In [ ]:
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

# This is the scenario that we can try to compile
$D= \sum_i B_i Z_i$
and Group commutator

In [102]:
from qibo import symbols, hamiltonians
b_list = np.linspace(-1,1,config["nqubits"]) 
b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(config["nqubits"]),b_list)]))
dm = d.dense.matrix

[Qibo 0.2.9|WARNING|2024-06-06 16:18:02]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


Sam prepared this function to check if the sign is correct. This we should use in eventual hpc training

In [ ]:
from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator,
)

In [103]:
qibo.models.dbi.utils.cs_angle_sgn(dbi, dm)

-1.0

In [104]:
for t in np.linspace(.005,.01,5):
    print (t)
    dbi(t,d=-dm)
    print(dbi.h.expectation(zero_state_t)-ene1)
    dbi.h = deepcopy(h_vqe_ham)

0.005
-0.000261644912622927
0.00625


## We need to schedule unitaries such as this one below

This later needs to be generalized to the GC3 PR from Andrew to get the same performance



In [72]:
def u_gcr(h,d,t):
    ud =  d.exp(t)
    u = h.exp(-t)
    return     ud.conj().T @ u @ud


In [73]:
for t in np.linspace(1e-9,1e-6,3):
    u_gcrd = u_gcr(h_vqe_ham,d,t)
    print(h_vqe_ham.expectation(u_gcrd@zero_state_t)-ene1)

-14.560174594055578
-14.560174593661694
-14.560174592480168


- This doesn't work now. It's better to have the correct expose dbr unitary and then take the exposed unitaries and compare to compiled circuits.


If someone picks it up:
- please try to find the correct scheduling as above
- then expose the unitaries
- and then use the XXZ decomposition function to replace h.exp(-t) by a circuit